In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import requests
from time import sleep

In [2]:
version = "1.0.1"

# Get the article list

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait, Select

from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)
driver.set_window_size(1920, 1080)

In [4]:
def gedjournalarticles(journal, filename ):
    start_url = "https://www.sciencedirect.com/search"
    driver.get(start_url)
    print(driver.current_url)
    sleep(4)
    
    content = driver.page_source
    
    # open old file
    olddf = pd.DataFrame({'url': [], 'journal': [] })
    try:
        olddf = pd.read_csv(filename, sep="|")
        print(olddf.tail())
    except:
        pass

    # set journal
    # publisher = driver.find_element_by_id("docId")
    # publisher.send_keys(journal['eissn'])
    publisher = driver.find_element_by_id("pub")
    publisher.send_keys(journal['title'])
    # just for Research articles
    try:
        driver.find_element_by_id("adv-search-FLA").click()
    except Exception as e:
        print(e)        
    
    sleep(4)
    # send request
    found = False
    for sl in driver.find_elements_by_class_name('search-row'):
        for search in sl.find_elements_by_tag_name('button'):
            if search.text == "Search":
                search.click()
                sleep(5)
                found = True
        if found:
            break    
    print(driver.current_url)
    
    articles = []
    def pagefind():
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # found the Article
        for adiv in soup.find_all('div', {'class': ['result-item-container']}):
            # check it is open access research article 
            divtype = adiv.find("div", {'class': ['OpenAccessArchive']}).get_text()
            if re.search('Open access', divtype ) and re.search('Research article', divtype ):
                for h2 in adiv.find_all("h2"):
                    # get link 
                    for a in h2.find_all("a"):
                        # filter out "Editorial Board"
                        if a.get_text() != "Editorial Board":
                            articles.append("https://www.sciencedirect.com"+a['href'])
    
        # try to find modal and close it
        for modal in driver.find_elements_by_class_name('modal-close-button'):
            modal.click()
            sleep(2)
        
        # avoid feedback bottom
        driver.set_window_size(1920, 1080)
    
        for pl in driver.find_elements_by_class_name('pagination-link'):        
            for a in pl.find_elements_by_tag_name('a'):
                if a.text == "next":
                    a.click()            
                    sleep(2)
                    return True

    # itterate over search pages on the Next bottom
    nextpage = True
    offset = 0
    while  nextpage:
        nextpage = pagefind()
        print(driver.current_url, end='\r')
        
    # filer out old articles
    oldarticles = olddf['url']
    articles = list( set(articles).difference(set(oldarticles) ))
    print(len(articles), "new article found!")    
    
    df = pd.DataFrame({'url': articles, 'journal_title': journal['title'], 
                       'journal_eissn': journal['eissn'],
                       'category': journal['category'] })
    
    return df, olddf

# Process article

In [5]:
import json

def getarticledetails(df, olddf, filename):

    articles = df['url'].values
    
    titles = [ None for _ in range(len(df))]
    abstracts = [ None for _ in range(len(df))]
    writers = [ None for _ in range(len(df))]
    dates = [ None for _ in range(len(df))]
    dois = [  None for _ in range(len(df)) ]
    keywords = [  None for _ in range(len(df)) ]

    for idx in range(len(articles)):
        # print percentiage of the process
        print( str(np.round(100*idx/len(df),2))+"%" , end='\r')
    
        url = df.iloc[idx]['url']
        driver.get(url)
        sleep(5)
        content = driver.page_source
    
        page_soup = BeautifulSoup(content, features="lxml", from_encoding='utf-8') 
    
        # abstract
        try:
            abstractdiv = page_soup.find(lambda tag: tag.name == 'div' and tag['class'] == ['abstract', 'author', ''])
            abstracts[idx] = abstractdiv.find("p").text
        except Exception as e:
            print("Error: ", url)
            print(e)
            break
            continue

        # title, doi and publication date
        for m in page_soup.find_all("meta"):
            if m.has_attr("name"):
                if m['name'] == "citation_title":
                    titles[idx] = m['content']            
                elif m['name'] == "citation_online_date":
                    # change to ISO dateformat
                    dates[idx] = datetime.datetime.strptime(m['content'], "%Y/%m/%d").strftime('%Y-%m-%d')
                elif m['name'] == "citation_doi":
                    dois[idx] = m['content']    

        # keywords
        key = []
        try:
            for kdiv in page_soup.find("div", {'class':['keywords-section']}).find_all("div", {'class':['keyword']}):
                key.append(kdiv.get_text())
        except:
            pass
        if len(key) > 0:
            keywords[idx] =  "#".join(key)
                    
        #  language
        try:
            for script in page_soup.find_all("script"):
                if re.search("abstracts", str(script.string)):
                    data = json.loads(script.string)
                    language = data['abstracts']['content'][0]['$']['lang']
        except:
            pass
    
        # writers
        writter = []
        aut = driver.find_element_by_id('author-group')
        for auta in aut.find_elements_by_class_name("workspace-trigger"):
            auta.click()
            sleep(1)
            # refresh page
            content = driver.page_source
            page_soup = BeautifulSoup(content, features="lxml", from_encoding='utf-8')
            auta.click()
            # refresh page
            content = driver.page_source
            page_soup = BeautifulSoup(content, features="lxml", from_encoding='utf-8')            
            for autdiv in page_soup.find_all("div", {'class':['WorkspaceAuthor']}):
                givenname = ""
                for aut_span in autdiv.find_all("span", {'class':['given-name']}):
                    givenname = aut_span.text
                    break
                familyname = ""
                for aut_span in autdiv.find_all("span", {'class':['surname']}):
                    familyname = aut_span.text
                    break
                institude = ""
                for aut_span in autdiv.find_all("div", {'class':['affiliation']}):
                    institude = aut_span.text
                    break
                writter.append(familyname+", "+givenname + "--"+institude)
        if len(writter) > 0:
            writers[idx] =  "#".join(writter)
        

    df['title'] = titles
    df['doi'] = dois
    df['abstract'] = abstracts
    df['writer'] = writers
    df['publishdate'] = dates
    df['keyword'] = keywords
    
    # merge with old df
    df = pd.concat([df, olddf])
    
    # save data
    df.to_csv(filename, sep="|", index=False)
                
    test = pd.read_csv(filename, sep="|")
    print(test.tail())

# Main

In [6]:
journals = [ 
    {'title':"Climate Risk Management", 'eissn': '2212-0963', 'category': 'Earth Science' },
    { 'title': 'Materials & Design', 'eissn': '1873-4197', 'category': 'Industrial'},
]


In [7]:
journals = [    
    { 'title': 'Engineering Science and Technology, an International Journal', 'eissn': '2215-0986', 'category': 'Civil Engineering'}
]

In [8]:
jidx = 0
filename = filename = 'data/journal_'+ journals[jidx]['title'].replace(" ", "_").replace(":", "").replace("&", "and") +'_'+version+'.csv'
df, olddf = gedjournalarticles( journals[jidx], filename )
getarticledetails(df, olddf, filename)

https://www.sciencedirect.com/search


NoSuchElementException: Message: Unable to locate element: [id="pub"]


In [ ]:
for jidx in range(len(journals)):
    filename = filename = 'data/journal_'+ journals[jidx]['title'].replace(" ", "_").replace(":", "").replace("&", "and") +'_'+version+'.csv'
    
    # search for articles
    df, olddf = gedjournalarticles( journals[jidx], filename )
    
    # get the articles details
    getarticledetails(df, olddf, filename)